In [ ]:
# Load Library
from ai_core_sdk.ai_core_v2_client import AICoreV2Client

# Create Connection
ai_core_client = AICoreV2Client(
    base_url = "https://api.ai.prod.us-east-1.aws.ml.hana.ondemand.com" + "/v2", # The present SAP AI Core API version is 2
    auth_url=  "https://eyts-developer-z8opke6n.authentication.us10.hana.ondemand.com" + "/oauth/token", # Suffix to add
    client_id = YOUR_CLIENT_ID,
    client_secret = YOUR_CLIENT_SECRET
)

In [ ]:
'''# Create object Store secret
response = ai_core_client.object_store_secrets.create(
    name = "s3housepricedc", # identifier for this secret within your SAP AI Core
    path_prefix = "example-dataset/house-price-toy", # path that we want to limit restrict this secret access to
    type = "S3",
    data = { # Dictionary of credentials of AWS
        "AWS_ACCESS_KEY_ID": YOUR_ACCESS_KEY,
        "AWS_SECRET_ACCESS_KEY": YOUR_ACCESS_SECRET
    },
    bucket = "sap-aicore-test", # Edit this
    region = "us-east-1", # Edit this
    endpoint = "s3.amazonaws.com", # Edit this
    resource_group = "default" # object store secret are restricted within this resource group. you may change this when creating secret for another resource group.
)
print(response.__dict__)
'''

In [ ]:
'''# Create Artifact
from ai_api_client_sdk.models.artifact import Artifact
from ai_api_client_sdk.models.label import Label

response = ai_core_client.artifact.create(
    name = "House Price Dataset DC v2", # Custom Non-unqiue identifier
    kind = Artifact.Kind.DATASET,
    url = "ai://s3housepricedc/data/jan", #
    scenario_id = "learning-datalines-dc",
    description = "Prices in the month of Jan",
    labels = [
        Label(key="ext.ai.sap.com/month", value="Jan"), # any descriptive key-value pair, helps in filtering, key must have the prefix ext.ai.sap.com/
    ],
    resource_group = "default" # required to restrict object store secret usage within a resource group
)

print(response.__dict__)'''

In [ ]:
'''# Create Artifact (Feb)
from ai_api_client_sdk.models.artifact import Artifact
from ai_api_client_sdk.models.label import Label

response = ai_core_client.artifact.create(
    name = "House Price Dataset DC v2 Feb", # Custom Non-unqiue identifier
    kind = Artifact.Kind.DATASET,
    url = "ai://s3housepricedc/data/feb", #
    scenario_id = "learning-datalines-dc",
    description = "Prices in the month of Feb",
    labels = [
        Label(key="ext.ai.sap.com/month", value="Feb"), # any descriptive key-value pair, helps in filtering, key must have the prefix ext.ai.sap.com/
    ],
    resource_group = "default" # required to restrict object store secret usage within a resource group
)

print(response.__dict__)'''

In [ ]:
'''#Use Artifacts with workflows using a configuration
from ai_api_client_sdk.models.parameter_binding import ParameterBinding
from ai_api_client_sdk.models.input_artifact_binding import InputArtifactBinding

response = ai_core_client.configuration.create(
    name = "House Price January 1",
    scenario_id = "learning-datalines",
    executable_id = "data-pipeline",
    input_artifact_bindings = [
        InputArtifactBinding(key = "housedataset", artifact_id = "<YOUR_JAN_ARTIFACT_ID>") # placeholder as name
    ],
    parameter_bindings = [
        ParameterBinding(key = "DT_MAX_DEPTH", value = "3") # placeholder name as key
    ],
    resource_group = "default"
)
print(response.__dict__)
'''

In [ ]:
# Query Deployment Status
DEPLOYMENT_ID = "d575402dfd33b2cd"

response = ai_core_client.deployment.get(
    deployment_id= DEPLOYMENT_ID,
    resource_group="default"
)

print('Status: ' + str(response.status) + '\nStart Time: ' + str(response.__dict__['start_time']) + '\nCompletion Time: ' + str(response.__dict__['completion_time']))
DEPLOYMENT_URL = str(response.__dict__['deployment_url'])

In [ ]:
#Make Prediction using PYTHON POST REST
import requests

# URL
deployment_url = DEPLOYMENT_URL

# Preparing the input for inference
test_input = {
    "MedInc": 8.3252,
    "HouseAge": 41,
    "AveRooms": 6.984126984,
    "AveBedrms":1.023809524,
    "Population":322,
    "AveOccup": 2.555555556,
    "Latitude":37.88,
    "Longitude":-122.23
}

endpoint = f"{deployment_url}/v2/predict" # endpoint implemented in serving engine
headers = {"Authorization": ai_core_client.rest_client.get_token(),
           'ai-resource-group': "default",
           "Content-Type": "application/json"}
response = requests.post(endpoint, headers=headers, json=test_input)

print('Inference result: $', response.json()[0]*100000)
